# 데이터 전처리
> 수집한 데이터를 머신러닝 알고리즘을 운용할 수 있는 형태로 가공하는 작업을 지칭합니다.  
데이터의 분할, 결측치, 범주형 데이터, 클래스 불균형 등 일반적인 데이터에서 발생 가능한 문제를 해결하는 일반적인 방법론을 지칭 하기도 합니다.  
다만 데이터의 특성 및 사용해야 하는 모델 케이스에 따라 수행해야하는 전처리 과정이 달라 기술적인 어려움이 있습니다.  
이번 시간에는 기본적인 전처리 방법들을 소개하고 샘플데이터에 적용하며 전처리 방법을 익혀 보겠습니다.  

## titanic 데이터 설명
> 타이타닉호에 승선했던 탑승자 정보를 담고 있는 데이터입니다. 데이터분석 필드에서는 교육용 및 입문용 데이터로 오랫동안 사랑받아 온 데이터이며  
탑승자 정보를 바탕으로 이 탑승자가 생존했는지 혹은 사망했는지 분류하는 모델링을 목적으로 제작 된 데이터입니다.  

**columns 정보**
- pclass : 객실 등급  
- survived : 생존 유무
- name : 칭호를 포함한 이름
- sex : 성별
- age : 나이
- sibsp : 형제 혹은 부부의 수
- parch : 부모, 혹은 자녀의 수
- ticket : 탑승권 종류 (각 숫자가 의미하는 바는 데이터를 통해 알 수 있는 것이 없을 수 있습니다.)
- fare : 지불한 운임
- cabin : 객실정보
- embarked : 선착장정보
- boat : 탈출한 보트가 있다면 boat 번호
- body : 사망자의 시신 수습 후 부여한 일련번호
- home : 출신

## 결측 데이터 처리

In [ ]:
# 필요모듈 import
import pandas as pd

In [ ]:
# 타이타닉 데이터 로드
df = pd.read_csv('./data/titanic.csv')
df.head()

In [ ]:
import seaborn as sns

In [ ]:
# 결측 데이터 분포 시각화
sns.heatmap(df.isna())
# 상관관계분석
# sns.heatmap(df.corr())

이 중 cabin, home.dest 컬럼 데이터의 경우 결측치 비중이 높고 주변 데이터로부터 빈 데이터를 유추할 수 있는 힌트가 없어 삭제 처리 합니다.

In [ ]:
# 여러가지 방법으로 컬럼 삭제
df.drop('cabin', axis=1, inplace=True)

In [ ]:
del df['home.dest']

In [ ]:
df.pop('ticket')

In [ ]:
df.info()

In [ ]:
# 결측치 비율 계산
df['age'].isna().sum() / len(df) * 100

In [ ]:
# age
# 연속형 데이터의 결측치 처리 ---> 평균, 중앙값
# fillna(x) 전달하는 x값으로 결측치를 치환
df['age'].fillna(df['age'].median(), inplace=True)

In [ ]:
df['fare'].fillna(df['fare'].mean(), inplace=True)

In [ ]:
# 컬럼 고윳값 카운트
# 컬럼의 고윳값을 계산해서 내림차순으로 정렬 함수
print(df['embarked'].unique()) # 고윳값 출력
df['embarked'].value_counts() # 고윳값의 갯수를 세어서 내림차순으로 정렬

In [ ]:
# age, fare, embarked 컬럼의 빈칸 채우기
# age 결측치 비율이 높지 않고 유의수준 안의 샘플 갯수이기에 평균값으로 결측치를 채워넣습니다.
df['embarked'].fillna('S', inplace=True)

boat, body의 경우 유추하여 값을 채워 넣을 수 있는 데이터는 없지만 데이터의 존재 자체로서 생존 여부를 판단 할 수 있는 데이터이기에 데이터를 명목화 시키겠습니다.

## 범주형 변수의 처리
> 카테고리컬 데이터라고도 하며 데이터가 특정 구간에 속하여 구분이 가능한 데이터 분포를 보이는 변수의 경우 범주형 데이터라 한다.  
범주형 데이터의 경우 보통 문자열 구분이 되어있어 이를 컴퓨터가 인식 가능한 숫자형태로 바꿔주는데 더미화, 혹은 원핫인코딩이라는 형태로 변경한다.  

In [ ]:
# 보트 컬럼의 결측치 여부를 이진형 데이터로 치환
df['body'] = df['body'].isna()

In [ ]:
df['boat'] = df['boat'].isna()

In [ ]:
import numpy as np
# 넘파이 브로드캐스팅
np.array([1, 2, 3, 4]) > 4

In [ ]:
df['survived'] == 1

In [ ]:
# 데이터 선택자 iloc, loc[], 행 / 열 기준으로 인덱스 및 슬라이싱 코드를 전달  
df.iloc[0:10, 0:5] # 무조건 인덱스 넘버만 가능
df.loc[df['survived'] == 1, ['pclass', 'survived', 'name', 'sex', 'age']] # 문자 전달도 가능

In [ ]:
df.loc[df['age'] < 10]

In [ ]:
# 컬럼데이터가 True면 1, False면 0 으로 변환하는 커스텀함수 제작
def make_binary(x):
    if x == True:
        return 1
    else:
        return 0

In [ ]:
# 컬럼데이터에 위에서 제작한 커스텀함수 적용
df['body'] = df['body'].apply(make_binary)

In [ ]:
df['boat'] = df['boat'].apply(make_binary)

In [ ]:
df.info()

### 카테고리가 여러개인 변수 처리

In [ ]:
df.head()

이름의 경우 미들네임에 따라 생존에 영향을 줄 수 있는 경우가 있습니다. 전체 미들네임에 해당하는 카테고리를 선택하지 않고 일부만을 데이터화 시킵니다.

In [ ]:
# name 컬럼 첫번째 인덱스 데이터 선택해서 test_text로 저장
test_text = df.loc[0, 'name']

In [ ]:
# 선별텍스트에서 문자열만 선택
test_text.split()[1]

In [ ]:
# 미들네임 선택 함수
def make_title(x):
    return x.split()[1]

In [ ]:
make_title(test_text)

In [ ]:
# 미들네임 선택 함수 적용 후 name 컬럼 원본값 변경
df['name'] = df['name'].apply(make_title)

In [ ]:
# name 컬럼의 문자열 그대로는 학습에 사용이 불가능하기에 이를 다시 숫자형태로 치환 + 기타 미들네임을 0으로 변환
def make_name(x):
    if x == 'Mrs.':
        return 1
    elif x == 'Miss.':
        return 2
    elif x == 'Mr.':
        return 3
    elif x == 'Master.':
        return 4
    else:
        return 0

In [ ]:
# 커스텀함수 적용
df['name'] = df['name'].apply(make_name)

In [ ]:
df.head()

In [ ]:
# 데이터프레임 내부에 카테고리컬 컬럼을 원핫인코딩 변환 작업
df = pd.get_dummies(df, columns=['pclass', 'name', 'sex', 'embarked'], drop_first=True)

## 타겟, 학습데이터 분할

In [ ]:
# 타겟데이터 분할

# 학습데이터 분할


## 스케일 조정
> 사용해야 하는 머신러닝 모델의 특징에 따라 학습 및 테스트에 필요한 데이터의 스케일 보정이 필요한 경우가 있습니다.  
이는 스케일이 큰 설명변수의 영향력이 비대해지는 문제를 방지하기 위해 사용합니다.

가장 일반적으로 사용하는 스케일 보정 방법론 두가지를 사용해보겠습니다.

> - normalization
>> 설명변수의 범위를 0~1 사이로 조정
> - standardization  
>> 설명변수의 범위를 평균 0, 분산이 1이 되도록 조정, 정규분포를 표준 정규분포화 시키는 것과 같음.

In [ ]:
# 스케일러 모델 로딩


In [ ]:
# 모델 생성


In [ ]:
# 모델 학습


In [ ]:
# 원 데이터 변환


In [ ]:
# 변환 데이터 확인


In [ ]:
# test 데이터에 적용


## 클래스 불균형 (imbalance data)
> 모델링 프로젝트를 진행하는 도중 실제 문제에 굉장히 빈번하게 발생하지만 해결이 어려운 문제가 클래스 불균형 문제입니다.  
이는 학습에 필요한 충분한 클래스의 샘플 수를 확보하지 못해 모델의 결과값이 샘플 수가 큰 클래스로 편향되는 문제를 발생 시킵니다.  
ex) 1000개 샘플 중 10개의 오류를 모두 정상이라 판별해도 acc는 99%입니다.  
타겟 데이터의 불균형 문제를 해결하기 위한 방법으로 oversampling / undersampling을 활용 할 수 있습니다.

> - oversampling
>> 타겟데이터 클래스 샘플 수를 크기가 큰 타겟데이터 클래스 샘플 수와 같은 갯수로 생성
> - undersampling
>> 타겟데이터 클래스 샘플 수를 크기가 작은 타겟데이터 클래스 샘플 수와 같게 샘플 삭제

In [ ]:
# imblearn 설치
!pip install -U imbalanced-learn

In [ ]:
# SMOTE import
from imblearn.over_sampling import SMOTE

In [ ]:
# oversampling 적용
# imblearn oversampling SMOTE (Synthetic Minority Oversampling Technique)


## 모델학습

In [ ]:
# 모델 생성


In [ ]:
# 모델 학습


In [ ]:
# 예측값 저장


In [ ]:
# 분류평가표 출력


## datetime 형식 변환

In [ ]:
df1 = pd.read_csv('./data/energy1.csv', encoding='cp949')
df1.head()

In [ ]:
df1['num'].unique()

In [ ]:
len(df1.loc[df1['num'] == 1])

In [ ]:
df1.info()

In [ ]:
df1['date_time'] = pd.to_datetime(df1['date_time'])

In [ ]:
df1.loc[1, 'date_time'] - df1.loc[0, 'date_time']

In [ ]:
print(df1.loc[0, 'date_time'].year)
print(df1.loc[0, 'date_time'].month)
print(df1.loc[0, 'date_time'].day)
print(df1.loc[0, 'date_time'].hour)
print(df1.loc[0, 'date_time'].minute)
print(df1.loc[0, 'date_time'].second)
print(df1.loc[0, 'date_time'].microsecond)
print(df1.loc[0, 'date_time'].dayofweek) # 요일정보 0부터 월요일

In [ ]:
def make_day(x):
    return x.day

def make_weekofday(x):
    return x.dayofweek

def make_hour(x):
    return x.hour

In [ ]:
df1['day'] = df1['date_time'].apply(make_day)

In [ ]:
df1['hour'] = df1['date_time'].apply(make_hour)

In [ ]:
df1['weekday'] = df1['date_time'].apply(make_weekofday)

In [ ]:
df1.head()

In [ ]:
df1.loc[df1['weekday'] == 0]